[Website](https://nigeriapropertycentre.com/for-sale/houses/showtype)

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import pandas as pd
import time
from tqdm import tqdm

Single Page Property Data Extraction

In [ ]:
path = "C:/Users/HP/Downloads/chromedriver-win64/chromedriver.exe"
url = "https://nigeriapropertycentre.com/for-sale/houses/showtype?page=1"

service = Service(path)
driver = webdriver.Chrome(service=service)

driver.get(url)
property_links = [link.get_attribute('href') for link in driver.find_elements(By.XPATH, '//a[@itemprop="url"]')]

properties = []

for link_url in property_links:
    driver.get(link_url)

    time.sleep(3)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html')

    rows = soup.find('table', class_= 'table table-bordered table-striped')

    
    for row in rows:
        line = row.find_all('td')
        fields = [h.text.strip() for h in line]
        

        table = {
            field.split(":")[0].strip(): field.split(":")[1].strip() 
            for field in fields if ":" in field # and len(field.split(":")) > 1
            }
        try:
            figure = soup.find_all('span', class_='price')[1]
            price = float(figure.attrs['content'])
            table['Price'] = price
        except:
            pass 
        try: 
            dollar = soup.find('span', class_='naira-equiv')
            equiv = float(dollar.text.split()[1].replace(',','')[1:])           
            table['Price'] = equiv
        except:
            pass

        address = soup.find('div', class_='col-sm-8 f15 property-details')
        location = address.text.strip().split(',')[-2:]
        city = location[0].strip()
        state = location[1].strip()
        table['District'] = city 
        table['State'] = state

        properties.append(table)

        # back = driver.find_element(By.XPATH, '//a[@class="underline"]')
        # back.click()
    # driver.back()
    time.sleep(2) 

# driver.quit()

In [ ]:
properties

Traditional Pagination Strategy for Iteration

In [ ]:
properties = []

for x in range(1,201):
    
    path = "C:/Users/HP/Downloads/chromedriver-win64/chromedriver.exe"
    
    url = f"https://nigeriapropertycentre.com/for-sale/houses/showtype?page={x}"

    service = Service(path)
    driver = webdriver.Chrome(service=service)

    driver.get(url)

    property_links = [link.get_attribute('href') for link in driver.find_elements(By.XPATH, '//a[@itemprop="url"]')]

    for link_url in property_links:
        driver.get(link_url)

        time.sleep(10)

        # try:
        #     WebDriverWait(driver,10).until(
        #         EC.visibility_of_element_located(By.XPATH, '//table[@class="table table-bordered table-striped"]')
        #     )
        # except Exception as e:
        #     print(f"Error waiting for table on {link_url}: {e}")
        #     continue

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html')

        rows = soup.find('table', class_= 'table table-bordered table-striped')

        
        for row in rows:
            line = row.find_all('td')
            fields = [h.text.strip() for h in line]
            

            table = {
                field.split(":")[0].strip(): field.split(":")[1].strip() 
                for field in fields if ":" in field # and len(field.split(":")) > 1
                }
            try:
                figure = soup.find_all('span', class_='price')[1]
                price = float(figure.attrs['content'])
                table['Price'] = price
            except:
                pass 
            try: 
                dollar = soup.find('span', class_='naira-equiv')
                equiv = float(dollar.text.split()[1].replace(',','')[1:])           
                table['Price'] = equiv
            except:
                pass

            address = soup.find('div', class_='col-sm-8 f15 property-details')
            location = address.text.strip().split()[-2:]
            city = location[0].replace(',','')
            state = location[1].strip()
            table['District'] = city 
            table['State'] = state
            
            properties.append(table)

            # back = driver.find_element(By.XPATH, '//a[@class="underline"]')
            # back.click()
        # driver.back()
        time.sleep(2) 

# driver.quit()
# print(f'Successfully scraped {x} pages')

Attempting to involve tqdm to help show the progress bar of pages scraped

In [1]:
service = Service(path)
driver = webdriver.Chrome(service=service)
path = "C:/Users/HP/Downloads/chromedriver-win64/chromedriver.exe"

properties = []

start_page = 1
end_page = 50

for x in tqdm(range(start_page, end_page), desc="Scraping Pages"):
    
    url = f"https://nigeriapropertycentre.com/for-sale/houses/showtype?page={x}"
    driver.get(url)

    property_links = [link.get_attribute('href') for link in driver.find_elements(By.XPATH, '//a[@itemprop="url"]')]

    for link_url in property_links:
        driver.get(link_url)

        time.sleep(10)

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html')

        rows = soup.find('table', class_= 'table table-bordered table-striped')

        try:
            for row in rows:
                line = row.find_all('td')
                fields = [h.text.strip() for h in line]
                

                table = {
                    field.split(":")[0].strip(): field.split(":")[1].strip() 
                    for field in fields if ":" in field # and len(field.split(":")) > 1
                    }
                try:
                    figure = soup.find_all('span', class_='price')[1]
                    price = float(figure.attrs['content'])
                    table['Price'] = price
                except:
                    pass 
                try: 
                    dollar = soup.find('span', class_='naira-equiv')
                    equiv = float(dollar.text.split()[1].replace(',','')[1:])           
                    table['Price'] = equiv
                except:
                    pass

                address = soup.find('div', class_='col-sm-8 f15 property-details')
                location = address.text.strip().split()[-2:]
                city = location[0].replace(',','')
                state = location[1].strip()
                table['District'] = city 
                table['State'] = state

                properties.append(table)
        except Exception as e:
            print(f"Error processing {link_url}: {e}")
            
        time.sleep(2) 

# driver.quit()
print(f"Successfully scraped {len(properties)} properties across {end_page - start_page} pages.")

NameError: name 'Service' is not defined

In [ ]:
service = Service(path)
driver = webdriver.Chrome(service=service)
path = "C:/Users/HP/Downloads/chromedriver-win64/chromedriver.exe"

properties = []

total_properties = 0 
property_links_all_pages = []

start_page = 1
end_page = 250

for x in range(start_page, end_page):
    
    url = f"https://nigeriapropertycentre.com/for-sale/houses/showtype?page={x}"
    driver.get(url)

    property_links = [link.get_attribute('href') for link in driver.find_elements(By.XPATH, '//a[@itemprop="url"]')]
    property_links_all_pages.extend(property_links) 

with tqdm(total=len(property_links_all_pages), desc="Scraping properties") as pbar:
    for link_url in property_links_all_pages:
        driver.get(link_url)

        time.sleep(10)

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html')

        rows = soup.find('table', class_= 'table table-bordered table-striped')

        
        for row in rows:
            line = row.find_all('td')
            fields = [h.text.strip() for h in line]
            

            table = {
                field.split(":")[0].strip(): field.split(":")[1].strip() 
                for field in fields if ":" in field # and len(field.split(":")) > 1
                }
            try:
                figure = soup.find_all('span', class_='price')[1]
                price = float(figure.attrs['content'])
                table['Price'] = price
            except:
                pass 
            try: 
                dollar = soup.find('span', class_='naira-equiv')
                equiv = float(dollar.text.split()[1].replace(',','')[1:])           
                table['Price'] = equiv
            except:
                pass

            address = soup.find('div', class_='col-sm-8 f15 property-details')
            location = address.text.strip().split()[-2:]
            city = location[0].replace(',','')
            state = location[1].strip()
            table['District'] = city 
            table['State'] = state

            properties.append(table)

            total_properties +=1
            pbar.update(1)

            # back = driver.find_element(By.XPATH, '//a[@class="underline"]')
            # back.click()
        # driver.back()
        time.sleep(2) 

# driver.quit()
print(f"Successfully scraped {len(properties)} properties across {end_page - start_page} pages.")

Using Selenium to scrape table containing property info for a particular property

In [ ]:
# path = "C:/Users/HP/Downloads/chromedriver-win64/chromedriver.exe"
url = "https://nigeriapropertycentre.com/for-sale/houses/semi-detached-duplexes/lagos/ikoyi/2598042-lovely-5bed-semi-detached-home-with-a-bq"

# service = Service(path)
# driver = webdriver.Chrome(service=service)

# driver.get(url)

response = requests.get(url)

# page_source = driver.page_source
soup = BeautifulSoup(response.text, 'html')

address = soup.find('div', class_='col-sm-8 f15 property-details')
location = address.text.strip().split()[-2:]
city = location[0].replace(',','')

print(city)

Ikoyi
